# Toy ML Problem for FL theories

## Regression with BNNs

In [ ]:
import numpy as np
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
import matplotlib.pyplot as plt
from tqdm import trange

In [ ]:
input_noise = 0.1

#### Data simulation

In [ ]:
x = torch.linspace(-2, 2, 500)
y = x.pow(5) -10* x.pow(1) + input_noise*torch.rand(x.size())
x = torch.unsqueeze(x, dim=1)
y = torch.unsqueeze(y, dim=1)
def clean_target(x):
    return x.pow(5) -10* x.pow(1) + input_noise/2
def target(x):
    return x.pow(5) -10* x.pow(1) +input_noise*torch.rand(x.size())
x_test = torch.linspace(-2, 2, 300)
y_test = target(x_test)

x_test = torch.unsqueeze(x_test, dim=1)
y_test = torch.unsqueeze(y_test, dim=1)

plt.scatter(x.data.numpy(), y.data.numpy(), s=10)
plt.scatter(x.data.numpy(), clean_target(x).numpy(), s=10)

plt.show()

#### Standard Model

In [ ]:
model = nn.Sequential(
    nn.Linear(in_features=1, out_features=100),
    nn.Tanh(),
    # nn.GELU(),
    nn.Linear(in_features=100, out_features=1),
)

In [ ]:
mse_loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
for step in range(2000):
    optimizer.zero_grad()
    output = model(x)
    loss = mse_loss(output, y)
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
        print('[%d] loss: %.3f' % (step, loss.item()))

In [ ]:
outs = model(x_test).data.numpy()

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(x_test.data.numpy(), outs ,color='navy',lw=3,label='Predicted Mean Model')

#plt.plot(x_test.data.numpy(),mean_values,color='darkorange')
plt.plot(x_test.data.numpy(),y_test.data.numpy(),'.',color='darkorange',markersize=4,label='Test set')
plt.plot(x_test.data.numpy(),clean_target(x_test).data.numpy(),color='green',markersize=4,label='Target function')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')

#### TorchBNN BNN

In [ ]:
model_bnn = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1, out_features=100),
    nn.Tanh(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=100, out_features=1),
)
mse_loss = nn.MSELoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model_bnn.parameters(), lr=0.01)

In [ ]:
for step in trange(2000):
    pre = model_bnn(x)
    mse = mse_loss(pre, y)
    kl = kl_loss(model_bnn)
    cost = mse + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
print('- MSE : %2.2f, KL : %2.2f' % (mse.item(), kl.item()), end="\r")

In [ ]:
models_result = np.array([model_bnn(x_test).data.numpy() for k in range(100)])
models_result = models_result[:,:,0]    
models_result = models_result.T
mean_values = np.array([models_result[i].mean() for i in range(len(models_result))])
std_values = np.array([models_result[i].std() for i in range(len(models_result))])

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(x_test.data.numpy(), mean_values,color='navy', lw=3,label='Predicted Mean Model')
plt.fill_between(x_test.data.numpy().T[0],mean_values-3.0*std_values,mean_values+3.0*std_values,alpha=0.2,color='navy',label='99.7% confidence interval')
#plt.plot(x_test.data.numpy(),mean_values,color='darkorange')
plt.plot(x_test.data.numpy(),y_test.data.numpy(),'.',color='darkorange',markersize=4,label='Test set')
plt.plot(x_test.data.numpy(),clean_target(x_test).data.numpy(),color='green',markersize=4,label='Target function')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')

In [ ]:
sigmas = {}
for k,v in model_bnn.named_parameters():
    if 'sigma' in k:
        sigmas[k] = v.detach().numpy().squeeze()

In [ ]:
mean_sigmas = {}
for k, sig in sigmas.items():
    if sig.ndim > 0:
        s_mean = np.sqrt(np.sum(np.exp(2*sig))/len(sig))
    else:
        s_mean = np.exp(sig)

    mean_sigmas[k] = s_mean
    print(s_mean)
    print('---')

In [ ]:
results = {}
results['noise=2'] = mean_sigmas

In [ ]:
results["noise=4"] = mean_sigmas

In [ ]:
results["noise=0.1"] = mean_sigmas

#### BLITZ BNN

In [ ]:
from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator
import torch.nn.functional as F

In [ ]:
@variational_estimator
class BayesianRegressor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        #self.linear = nn.Linear(input_dim, output_dim)
        self.blinear1 = BayesianLinear(input_dim, 100)
        self.blinear2 = BayesianLinear(100, output_dim)
        
    def forward(self, x):
        x_ = self.blinear1(x)
        x_ = F.tanh(x_)
        return self.blinear2(x_)


In [ ]:
regressor = BayesianRegressor(1, 1)
optimizer = torch.optim.Adam(regressor.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
for step in range(2000):    
    optimizer.zero_grad()
    loss = regressor.sample_elbo(inputs=x, labels=y, criterion=criterion, sample_nbr=1, complexity_cost_weight=0.01)
    loss.backward()
    optimizer.step()
    print(f'Loss: {loss.item()}, iter: {step}', end="\r")

In [ ]:
models_result = np.array([regressor(x_test).data.numpy() for k in range(100)])
models_result = models_result[:,:,0]    
models_result = models_result.T
mean_values = np.array([models_result[i].mean() for i in range(len(models_result))])
std_values = np.array([models_result[i].std() for i in range(len(models_result))])

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(x_test.data.numpy(), mean_values,color='navy', lw=3,label='Predicted Mean Model')
plt.fill_between(x_test.data.numpy().T[0],mean_values-3.0*std_values,mean_values+3.0*std_values,alpha=0.2,color='navy',label='99.7% confidence interval')
#plt.plot(x_test.data.numpy(),mean_values,color='darkorange')
plt.plot(x_test.data.numpy(),y_test.data.numpy(),'.',color='darkorange',markersize=4,label='Test set')
plt.plot(x_test.data.numpy(),clean_target(x_test).data.numpy(),color='green',markersize=4,label='Target function')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')

## Classification with Iris